Submission File:

For each id in the test set, you must predict the value for the target Strength. 
The file should contain a header and have the following format:

id, Strength

5439, 55.2

5440, 12.3

5441, 83.4

...

In [12]:
# installs and imports here (Sklearn, matplotlib, numpy, pandas, etc.)
%pip install scikit-learn 
%pip install pandas
%pip install xgboost
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Data Preparation

# Put all the data we are going to use into a df. Puts target variable column into 'y'. 'X' is all data except 'y' column. Splits the data.
data_df = pd.read_csv('./data.csv', index_col='id')
y = data_df.Strength
X = data_df.drop('Strength', axis=1)
# Convert columns with mix of 0's and numbers into  binary, to see the effects. Actually made it a bit better.
#binary_columns = ['BlastFurnaceSlag', 'FlyAshComponent', 'SuperplasticizerComponent']
#for col in binary_columns:
    #X[col] = X[col].apply(lambda x: 1 if x > 0 else 0)

X

,CementComponent,BlastFurnaceSlag,FlyAshComponent,WaterComponent,SuperplasticizerComponent,CoarseAggregateComponent,FineAggregateComponent,AgeInDays
id,,,,,,,,
0,525.0,0.0,0.0,186.0,0.0,1125.0,613.0,3
1,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28
2,289.0,134.7,0.0,185.7,0.0,1075.0,795.3,28
3,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365
4,157.0,236.0,0.0,192.0,0.0,935.4,781.2,90
...,...,...,...,...,...,...,...,...
5402,446.0,24.0,79.0,162.0,11.6,967.0,712.0,3
5403,350.0,0.0,0.0,203.0,0.0,974.0,775.0,180
5404,295.8,0.0,0.0,185.7,0.0,1076.2,759.3,28


In [ ]:
# Trying to find the best parameters for the random forest model while avoiding overfitting.
param_grid = {
    'n_estimators': [100, 200, 400],
    'max_depth': [4, 6, 8],
    'min_samples_split': [8, 10, 12],
    'min_samples_leaf': [6, 8, 10]
}

# Create a RandomForestRegressor model instance
rf_model = RandomForestRegressor()

# Create the GridSearchCV instance
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5, n_jobs=4)

# Fit the GridSearchCV instance to the data
grid_search.fit(X, y)

# Print the best hyperparameters and the corresponding mean RMSE
best_params = grid_search.best_params_
best_score = -grid_search.best_score_
print(f"Best hyperparameters for the Random Forest model: {best_params}")
print(f"Mean RMSE for the Random Forest model with the best hyperparameters: {best_score}")

# {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200} yields RMSE 12.26
# {'max_depth': 5, 'min_samples_leaf': 8, 'min_samples_split': 15, 'n_estimators': 100} yields RMSE 12.16
# {'max_depth': 7, 'min_samples_leaf': 8, 'min_samples_split': 10, 'n_estimators': 200} yields RMSE 12.154 
# {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 10, 'n_estimators': 200} yields RMSE 12.14688
# {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 9, 'n_estimators': 200} yields RMSE 12.150
# {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 10, 'n_estimators': 200} yields RMSE 12.14188
# {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 10, 'n_estimators': 400} yields RMSE 12.1503
# {'max_depth': 4, 'min_samples_leaf': 8, 'min_samples_split': 10, 'n_estimators': 400} yields RMSE 12.2416

In [35]:
# Define the hyperparameter search space for XGBoost
param_grid = {
    'n_estimators': [1600],
    'max_depth': [4],
    'learning_rate': [0.005],
    'subsample': [0.7],
    'colsample_bytree': [0.4],
    'gamma': [0.1]
}

# Create an XGBRegressor model instance
xgb_model = XGBRegressor()

# Create the GridSearchCV instance
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5, n_jobs=4)

# Fit the GridSearchCV instance to the data
grid_search.fit(X, y)

# Print the best hyperparameters and the corresponding mean RMSE
best_params = grid_search.best_params_
best_score = -grid_search.best_score_
print(f"Best hyperparameters for the XGBoost model: {best_params}")
print(f"Mean RMSE for the XGBoost model with the best hyperparameters: {best_score}")

# {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200, 'subsample': 1} yields RMSE 12.258
# {'colsample_bytree': 0.35, 'gamma': 0, 'learning_rate': 0.025, 'max_depth': 6, 'n_estimators': 400, 'subsample': 0.75} yields RMSE 12.28
# {'colsample_bytree': 0.42, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 800, 'subsample': 0.75} yields RMSE 12.2175
# {'colsample_bytree': 0.38, 'gamma': 0.05, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 800, 'subsample': 0.68} yields RMSE 12.2167
# {'colsample_bytree': 0.4, 'gamma': 0.1, 'learning_rate': 0.005, 'max_depth': 6, 'n_estimators': 800, 'subsample': 0.7} yields RMSE 12.365
# {'colsample_bytree': 0.4, 'gamma': 0.1, 'learning_rate': 0.005, 'max_depth': 5, 'n_estimators': 1600, 'subsample': 0.7} yields RMSE 12.1586
# {'colsample_bytree': 0.4, 'gamma': 0.1, 'learning_rate': 0.005, 'max_depth': 4, 'n_estimators': 1600, 'subsample': 0.7} yields RMSE 12.127

Best hyperparameters for the XGBoost model: {'colsample_bytree': 0.4, 'gamma': 0.1, 'learning_rate': 0.005, 'max_depth': 4, 'n_estimators': 1600, 'subsample': 0.7}
Mean RMSE for the XGBoost model with the best hyperparameters: 12.127089570489332


In [21]:
# Create model instances of linear regression, random forrest, and XGBoost.
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=400, max_depth=5, min_samples_split=10, min_samples_leaf=8),
    'XGBoost': XGBRegressor(n_estimators=1600, max_depth=4, learning_rate=0.005, subsample=0.7, colsample_bytree=0.4, gamma=0.1)
}

In [22]:
# Loop through the models and perform cross-validation and train/test split evaluations
for model_name, model in models.items():
    # Perform 5-fold cross-validation, calculate mean rmse
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
    mean_rmse = -np.mean(cv_scores)
    print(f"Mean RMSE for the {model_name} model with 5-fold cross-validation: {mean_rmse}")

    # Train the model with the training split data, make predictions, calculate rmse
    train_X, validation_X, train_y, validation_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=0)
    model.fit(train_X, train_y)
    validation_predictions = model.predict(validation_X)
    rmse = mean_squared_error(validation_y, validation_predictions, squared=False)
    print(f"RMSE for the {model_name} model by splitting the data: {rmse}")
    print()  # Add a newline for readability

Mean RMSE for the Linear Regression model with 5-fold cross-validation: 14.608373730656874
RMSE for the Linear Regression model by splitting the data: 14.269586411578604

Mean RMSE for the Random Forest model with 5-fold cross-validation: 12.171904102613812
RMSE for the Random Forest model by splitting the data: 11.900186991628125

Mean RMSE for the XGBoost model with 5-fold cross-validation: 12.127089570489332
RMSE for the XGBoost model by splitting the data: 11.838597343166393



In [23]:
# Put all the data we are going to use into a df.
test_df = pd.read_csv('./test.csv', index_col='id')
X_test = test_df
# Convert columns with mix of 0's and numbers into binary, to see the effects. Actually made it a bit better.
#binary_columns = ['BlastFurnaceSlag', 'FlyAshComponent', 'SuperplasticizerComponent']
#for col in binary_columns:
    #X_test[col] = X_test[col].apply(lambda x: 1 if x > 0 else 0)

In [25]:
xgboost_model = XGBRegressor(n_estimators=1600, max_depth=4, learning_rate=0.005, subsample=0.7, colsample_bytree=0.4, gamma=0.1)
xgboost_model.fit(X, y)
test_y_pred = xgboost_model.predict(X_test)

# Create a DataFrame with the test IDs and predictions
submission_df = pd.DataFrame({'id': X_test.index, 'Strength': test_y_pred})
# Save the DataFrame to a CSV file for submitting to Kaggle
submission_df.to_csv('submissionXGBnobinary.csv', index=False)

Ended up getting these scores which would have placed me in the top 53 out of 765 (For my XGB model with binary columns):

XGB: 12.27347
XGB no binary: 12.29858
RF: 12.3461
RF no binary: 12.32284